In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lower_Grade_Glioma"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Lower_Grade_Glioma/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Lower_Grade_Glioma/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Lower_Grade_Glioma/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Lower_Grade_Glioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. List all subdirectories in the TCGA root directory
subdirectories = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {subdirectories}")

# The target trait is Lower_Grade_Glioma
target_trait = trait.lower()  # "lower_grade_glioma"
print(f"Searching for datasets suitable for {target_trait} analysis...")

# Find the directory that matches our target trait
matching_dirs = [d for d in subdirectories if target_trait.replace('_', ' ') in d.lower()]

# If exact match for LGG exists, use it
lgg_dir = "TCGA_Lower_Grade_Glioma_(LGG)"
if lgg_dir in subdirectories:
    selected_dir = lgg_dir
    print(f"Selected directory: {selected_dir} for {target_trait} analysis")
elif matching_dirs:
    # Otherwise use the first matching directory
    selected_dir = matching_dirs[0]
    print(f"Selected directory: {selected_dir} for {target_trait} analysis")
else:
    # If no suitable directory found, check if there's a combined glioma dataset
    glioma_dirs = [d for d in subdirectories if "glioma" in d.lower()]
    if glioma_dirs:
        selected_dir = glioma_dirs[0]
        print(f"No exact match found. Selected alternative directory: {selected_dir}")
    else:
        print("No suitable directory found for Lower_Grade_Glioma analysis.")
        selected_dir = None

if selected_dir:
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and clinical data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print("No directory found for Lower_Grade_Glioma. Skipping this trait.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not is_gene_available or not is_trait_available:
    print("Skipping this trait as no suitable data was found in TCGA.")


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'animal_insect_allergy_history', 'animal_insect_allergy_types', 'asthma_history', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_performance_status_assessment', 'eastern_cancer_oncology_group', 'eczema_history', 'family_history_of_cancer', 'family_history_of_primary_brain_tumor', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_

### Step 2: Find Candidate Demographic Features

In [3]:
# Examine the list of column names to identify potential age and gender columns
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 
                       'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_age_of_food_allergy', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['gender']

# Get the clinical data path
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Lower_Grade_Glioma_(LGG)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Preview the age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Preview the gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Candidate age columns:")
print(candidate_age_cols)
print("\nAge columns preview:")
print(age_preview)

print("\nCandidate gender columns:")
print(candidate_gender_cols)
print("\nGender columns preview:")
print(gender_preview)


Candidate age columns:
['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_age_of_food_allergy', 'year_of_initial_pathologic_diagnosis']

Age columns preview:
{'age_at_initial_pathologic_diagnosis': [31.0, 67.0, 44.0, 37.0, 50.0], 'days_to_birth': [-11509.0, -24578.0, -16297.0, -13565.0, -18494.0], 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [nan, nan, nan, nan, nan], 'first_diagnosis_age_of_animal_insect_allergy': ['12 - 20 Years', nan, nan, nan, nan], 'first_diagnosis_age_of_food_allergy': [nan, nan, nan, nan, nan], 'year_of_initial_pathologic_diagnosis': [2005.0, 2005.0, 2006.0, 2009.0, 2010.0]}

Candidate gender columns:
['gender']

Gender columns preview:
{'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Inspect the candidate columns for age and gender

# For age, we have several candidates. Let's examine each:
# 'age_at_initial_pathologic_diagnosis' - Looks like actual age in years (good)
# 'days_to_birth' - Negative values, likely days from birth to some reference point (can be converted)
# 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust' - All NaN in preview, not useful
# 'first_diagnosis_age_of_animal_insect_allergy' - String format with ranges, mostly NaN, not ideal
# 'first_diagnosis_age_of_food_allergy' - All NaN in preview, not useful
# 'year_of_initial_pathologic_diagnosis' - Year of diagnosis, not patient's age

# For gender, we only have one candidate column:
# 'gender' - Contains 'MALE' and 'FEMALE' values which is suitable

# Choose the most appropriate columns
age_col = 'age_at_initial_pathologic_diagnosis'  # Direct age values in years
gender_col = 'gender'  # Clear gender information

# Print the chosen columns and their information
print(f"Chosen age column: {age_col}")
print(f"Age column preview: {{'age_at_initial_pathologic_diagnosis': [31.0, 67.0, 44.0, 37.0, 50.0]}}")

print(f"Chosen gender column: {gender_col}")
print(f"Gender column preview: {{'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}}")


Chosen age column: age_at_initial_pathologic_diagnosis
Age column preview: {'age_at_initial_pathologic_diagnosis': [31.0, 67.0, 44.0, 37.0, 50.0]}
Chosen gender column: gender
Gender column preview: {'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Lower_Grade_Glioma/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 530)
Clinical data saved to ../../output/preprocess/Lower_Grade_Glioma/clinical_data/TCGA.csv
Clinical data shape: (530, 3)
Number of samples in clinical data: 530
Number of samples in genetic data: 530
Number of common samples: 530
Linked data shape: (530, 19851)


Data shape after handling missing values: (530, 19851)
Quartiles for 'Lower_Grade_Glioma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Lower_Grade_Glioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 32.0
  50% (Median): 41.0
  75%: 53.0
Min: 14.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 238 occurrences. This represents 44.91% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset deemed not usable based on validation criteria. Data not saved.
Preprocessing completed.
